In [2]:
import requests
import warnings
import json
import time
import pandas as pd
warnings.filterwarnings("ignore")

In [3]:
def get_headers(cookie):
    '''
    根据cookies构建消息请求头
    '''  
    myheaders = {
        'Content-Type': 'application/json',
        'cookie': cookie,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36',
        'vpn-12-o2-uims.jlu.edu.cn':'',
    }
    return myheaders



def get_course(myheaders):
    '''
    return:
    pd.DataFrame.from_dict(course_list): 已选以及未选的课程列表
    course_map: 未选的课程以及编号，该编号用于选课
    '''
    
    course_html = 'https://vpns.jlu.edu.cn/https/77726476706e69737468656265737421e5fe4c8f693a6445300d8db9d6562d/ntms/service/res.do?vpn-12-o2-uims.jlu.edu.cn'
    deHtml = 'https://vpns.jlu.edu.cn/https/77726476706e69737468656265737421e5fe4c8f693a6445300d8db9d6562d/ntms/service/res.do?vpn-12-o2-uims.jlu.edu.cn'
    course_dict = {  
        'tag': "lessonSelectLog@selectStore", 
        'branch': "default", 
        'params': {'splanId': 20}   # 注意，此参数每学期不同会发生变化，不一定一直适用！！需要进行修改维护！！通过分析选课的请求可以得到！！
    }
    deInfo = {
        'tag': "lessonSelectLogTcm@selectGlobalStore",
        'branch': "self",
        'params': {'lslId':0, 'myCampus':"Y"}    # Y选课, N退课
    }
    type_dict = {3060:'必修课', 3061:'选修课', 3062:'限选课', 3063:'未知', 3064:'选修课', 3065:'校选修课'}
    course_list = []
    course_map = {}
    
    # 获取所有课程
    course_json = json.dumps(course_dict)
    course_result = requests.post(course_html, data = course_json, headers = myheaders, allow_redirects=True, verify=False)
    course_detail = json.loads(course_result.text)
    
    # 根据获取的json格式数据转存数据
    for i in range(len(course_detail['value'])):
        single = {}
        single['课程名称'] = course_detail['value'][i]['lesson']['courseInfo']['courName']
        single['选否'] = course_detail['value'][i]['selectResult']
        single['选课类型'] = type_dict[course_detail['value'][i]['applyPlanLesson']['selectType']]
        lslId = course_detail['value'][i]['lslId']
        deInfo['params']['lslId'] = lslId
        de_json = json.dumps(deInfo)
        de_result = requests.post(deHtml, data = de_json, headers = myheaders, allow_redirects=True, verify=False)
        de = json.loads(de_result.text)
        single['选课编号'] = de['value'][0]['lsltId']
        course_list.append(single)
        if course_detail['value'][i]['selectResult'] == 'N':
            course_map[single['选课编号']] = single['课程名称']    # 仅存储未选课程的编号，其他可以自行查看
        time.sleep(0.5)    # 调速，防止被封
    return pd.DataFrame.from_dict(course_list), course_map



def fuck_course(lsltId_list, course_map, myheaders):
    '''
    根据lsltId_list中的课程id进行抢课
    '''
    
    course_detail = {
        'lsltId': 0,    
        'opType': "Y"    # Y选课, N退课
    }
    course_html = 'https://vpns.jlu.edu.cn/https/77726476706e69737468656265737421e5fe4c8f693a6445300d8db9d6562d/ntms/action/select/select-lesson.do?vpn-12-o2-uims.jlu.edu.cn'
    i = 1
    while True:    # 抢课轮数，暂未设置所有课程抢课成功后终止，需要手动终止
        try:
            for lsltId in lsltId_list:    # 抢每一门课
                course_detail['lsltId'] = lsltId
                detail_json = json.dumps(course_detail)
                result = requests.post(course_html, data = detail_json, headers = myheaders, allow_redirects=True, verify=False)
                info = json.loads(result.text)     
                print(info)
                if info['count'] == 1:
                    print('第' + str(i) + '次：'+ str(course_map[str(lsltId)]) +'  抢课成功')    # 可用于捡漏
                else:
                    print('第' + str(i) + '次：'+ str(course_map[str(lsltId)]) +'  抢课失败  原因：'+ info['msg'])
                #time.sleep(1)    ######################  调速  ######################
            i+=1
        except Exception as e:    # 可能服务器突然没响应
            print(e)

In [1]:
# 下面为具体用法

In [4]:
cookie = 'wengine_vpn_ticket=................; refresh=1'    # ...为你自己的cookie内容

In [5]:
myheaders = get_headers(cookie)    # 获取请求头

In [6]:
df_course, course_map = get_course(myheaders)    # 获取待抢课程编号

In [7]:
print(course_map)    # 输出course_map，根据其内容选择要抢的课程所对应的编号

{'72243539': '托福与语言能力、策略培养', '72243874': '科学哲学概览', '72244134': '新媒体素养与传播', '72052829': '软件项目管理', '72052306': 'Windows程序设计', '71650363': '大数据技术与应用*', '71862761': '多媒体技术', '72137090': '线性规划', '71978271': '数据挖掘', '71701524': '生物信息学入门（双语）', '72134721': '虚拟现实技术基础', '71575961': '设计模式', '71846616': 'UML建模技术及应用', '71643243': '模糊数学与应用', '71566009': '数据库应用技术*', '71686416': '程序安全检测技术'}


In [ ]:
lsltId_list = [71575961, 71978271]
fuck_course(lsltId_list, course_map, myheaders)    # 开始抢课

{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第1次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第1次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第2次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第2次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第3次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第3次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第4次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第4次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第5次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'e

{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第38次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第39次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第39次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第40次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第40次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第41次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第41次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第42次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第42次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'cou

{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第77次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第78次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第78次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第79次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第79次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第80次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第80次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'count': 0, 'errno': 2080, 'items': None, 'msg': '班级人数已经到达上限', 'status': -12}
第81次：设计模式  抢课失败  原因：班级人数已经到达上限
{'count': 0, 'errno': 1995, 'items': None, 'msg': '上课时间冲突，不能选课', 'status': -6}
第81次：数据挖掘  抢课失败  原因：上课时间冲突，不能选课
{'cou

In [ ]:
######
######
######
# 测试用例以及部分错误，不必运行此块，仅供个人调试时参考
######
######
######


course_detail = {
    'lsltId': "70228928",    
    'opType': "N"    # Y选课, N退课
}
detail_json = json.dumps(course_detail)
course_html = 'https://vpns.jlu.edu.cn/https/77726476706e69737468656265737421e5fe4c8f693a6445300d8db9d6562d/ntms/action/select/select-lesson.do?vpn-12-o2-uims.jlu.edu.cn'
for i in range(1,3):
    try:
        result = requests.post(course_html, data = detail_json, headers = myheaders, allow_redirects=True, verify=False)
        info = json.loads(result.text)
        if info['count'] == 1:
            print('第' + str(i) + '次：抢课成功')    # 可用于捡漏
            break
        else:
            print('第' + str(i) + '次：失败  原因：'+ info['msg'])
    except Exception as e:    # 可能服务器突然没响应
        print(e)
    time.sleep(5)    # 自动调速

    
'''
{"count":0,"errno":2080,"items":null,"msg":"班级人数已经到达上限","status":-12}
{"count":1,"errno":0,"items":["S"],"msg":"","status":0}
{"count":0,"errno":1995,"items":null,"msg":"上课时间冲突，不能选课","status":-6}
{'count': 0, 'errno': -3, 'items': None, 'msg': '权限不足，您不能执行此操作或者获取数据', 'status': -3}
{'count': 0, 'errno': 1931, 'items': None, 'msg': '尚未开始选课或者阶段不对', 'status': -12}

{'applyPlanLesson': {'aplId': 524009,
  'planDetail': {'credit': 2},    # 学分
  'testForm': 3091,
  'selectType': 3065,    # 选课类型
  'isReselect': 'N'},    # 选否
 'notes': None,    # 选课反馈
 'selLssgCnt': 0,
 'replyTag': None,    # 反馈备注
 'selectResult': 'N',    # 选否
 'lslId': 50245283,
 'lesson': {'totalClassHour': 30,
  'teachSchool': {'schoolName': '通信工程学院'},
  'lessonId': 67787,
  'courseInfo': {'courType2': 3040, 'courseId': 27121, 'courName': '医用机器人专题'}},
 'sumLssgCnt': 1}
 ''' 